<a href="https://colab.research.google.com/github/imabari/ImabariScraping/blob/master/TEI_XML%E3%83%95%E3%82%A1%E3%82%A4%E3%83%AB%E3%81%8B%E3%82%89%E6%8A%9C%E3%81%8D%E5%87%BA%E3%81%97%E3%81%9F%E5%9C%B0%E7%90%86%E6%83%85%E5%A0%B1%E3%82%92%E5%9C%B0%E5%9B%B3%E4%B8%8A%E3%81%AB%E3%83%9E%E3%83%83%E3%83%94%E3%83%B3%E3%82%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ダウンロード

In [1]:
!curl -O "https://www.dhii.jp/dh/tei/soseki_letter_19000908.xml"
!curl -O "https://www.dhii.jp/dh/tei/soseki_letter_19001008.xml"
!curl -O "https://www.dhii.jp/dh/tei/soseki_letter_19001022.xml"

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 16262  100 16262    0     0   7795      0  0:00:02  0:00:02 --:--:--  7795
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 15417  100 15417    0     0  27142      0 --:--:-- --:--:-- --:--:-- 27094
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 18997  100 18997    0     0  32529      0 --:--:-- --:--:-- --:--:-- 32473


# スクレイピング

In [2]:
import pathlib
import pandas as pd

In [3]:
dfs = []

for p in pathlib.Path(".").glob("soseki_letter*.xml"):

    dfs.append(pd.read_xml(p, namespaces={"tei": "http://www.tei-c.org/ns/1.0"}, xpath="//tei:location|//tei:location/tei:address"))

In [4]:
df = pd.concat(dfs).reset_index(drop=True)
df

,placeName,geo,address,country,region,settlement,street
0,ペナン,5.418799642029674 100.345223003859,NaN,None,None,None,None
1,None,35.70348307829794 139.73306153078642,NaN,None,None,None,None
2,None,None,NaN,日本,東京,牛込區,矢来町三番地中ノ丸丙八十五号
3,Paris,48.86694190133614 2.2812760907779848,NaN,France,Paris,None,None
4,None,None,NaN,None,None,None,22 Rue Gustave Courbet
5,None,35.70348307829794 139.73306153078642,NaN,None,None,None,None
6,None,None,NaN,日本,東京,牛込區,矢来町三番地中ノ丸丙八十五号
7,アデン,12.791279269474849 44.985519313361195,NaN,None,None,None,None
8,None,35.70348307829794 139.73306153078642,NaN,None,None,None,None
9,None,None,NaN,日本,東京,牛込區,矢来町三番地中ノ丸丙八十五号


# 前処理

In [5]:
# placeNameが欠損の場合はregionで補完
df["placeName"].mask(df["placeName"].isna(), df["region"], inplace=True)

In [6]:
# geoを補完、placeNameが欠損の場合は除去
df = df.assign(geo=df["geo"].ffill()).loc[df["placeName"].notna()]

In [7]:
df

,placeName,geo,address,country,region,settlement,street
0,ペナン,5.418799642029674 100.345223003859,NaN,None,None,None,None
2,東京,35.70348307829794 139.73306153078642,NaN,日本,東京,牛込區,矢来町三番地中ノ丸丙八十五号
3,Paris,48.86694190133614 2.2812760907779848,NaN,France,Paris,None,None
6,東京,35.70348307829794 139.73306153078642,NaN,日本,東京,牛込區,矢来町三番地中ノ丸丙八十五号
7,アデン,12.791279269474849 44.985519313361195,NaN,None,None,None,None
9,東京,35.70348307829794 139.73306153078642,NaN,日本,東京,牛込區,矢来町三番地中ノ丸丙八十五号


In [8]:
# 重複削除
df.drop_duplicates(subset="geo", inplace=True)

In [9]:
# 緯度経度分割
df[["lat", "lon"]] = df["geo"].str.split(expand=True).astype(float)

In [10]:
df

,placeName,geo,address,country,region,settlement,street,lat,lon
0,ペナン,5.418799642029674 100.345223003859,NaN,None,None,None,None,5.418800,100.345223
2,東京,35.70348307829794 139.73306153078642,NaN,日本,東京,牛込區,矢来町三番地中ノ丸丙八十五号,35.703483,139.733062
3,Paris,48.86694190133614 2.2812760907779848,NaN,France,Paris,None,None,48.866942,2.281276
7,アデン,12.791279269474849 44.985519313361195,NaN,None,None,None,None,12.791279,44.985519


# 地図

In [11]:
import folium

In [12]:
map = folium.Map(
    location=[35.5, 138.5],
    zoom_start=2,
)

In [13]:
for i, r in df.iterrows():

    folium.Marker(
        location=[r.lat, r.lon],
        popup=folium.Popup(f'<p>{r.placeName}</p>', max_width=300),
    ).add_to(map)

In [14]:
map